In [40]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action = 'ignore')

In [19]:
df1 = pd.read_csv('../raw/2018/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2018.csv', encoding = 'euc-kr')
df2 = pd.read_csv('../raw/2018/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2018.csv', encoding = 'euc-kr')
df3 = pd.read_csv('../raw/2019/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2019.csv', encoding = 'euc-kr')
df4 = pd.read_csv('../raw/2019/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2019.csv', encoding = 'euc-kr')
df5 = pd.read_csv('../raw/2020/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2020.csv', encoding = 'euc-kr')
df6 = pd.read_csv('../raw/2020/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2020.csv', encoding = 'euc-kr')
df7 = pd.read_csv('../raw/2021/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2021.csv', encoding = 'euc-kr')
df8 = pd.read_csv('../raw/2021/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2021.csv', encoding = 'euc-kr')

In [20]:
#pcode에 선수이름을 붙이는 함수 작성
def merge_name(df1, df2):
    return df1.set_index('PCODE').join(df2[['PCODE', 'NAME']].set_index('PCODE'))

In [21]:
df_2018 = merge_name(df1, df2)
df_2019 = merge_name(df3, df4)
df_2020 = merge_name(df5, df6)
df_2021 = merge_name(df7, df8)

In [29]:
df_all = pd.concat([df_2018, df_2019, df_2020, df_2021]).reset_index()
df_all.head()

,PCODE,GYEAR,G_ID,PIT_ID,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM,NAME
0,60100,2018,20180328HHNC0,180328_203348,HH,7,148.19,-4.7,땅볼아웃,140.56,마산,백창수
1,60100,2018,20180330SKHH0,180330_190656,HH,2,153.79,5.8,직선타,142.41,대전,백창수
2,60100,2018,20180331SKHH0,180331_174531,HH,2,148.65,-13.5,땅볼아웃,135.50,대전,백창수
3,60100,2018,20180331SKHH0,180331_192505,HH,7,151.85,-3.7,병살타,131.71,대전,백창수
4,60100,2018,20180331SKHH0,180331_200628,HH,9,154.12,19.0,플라이,141.85,대전,백창수


In [58]:
#구장별 배럴타구 기준 적용하여 각 타구별로 배럴타구 여부 생성
def barrel(df):
    df['barrel'] = False
    df_tmp = df[['HIT_ANG_VER', 'HIT_VEL', 'HIT_RESULT', 'STADIUM']]
    for st in df_tmp['STADIUM'].unique():
        df_tmp2 = df_tmp[df_tmp['STADIUM'] == st].iloc[:, :-1]
        df_tmp3 = df_tmp2[df_tmp2['HIT_RESULT'].apply(lambda x : x in ['홈런', '2루타', '3루타'])]

        q1H_Ang, q3H_Ang = np.percentile(df_tmp3[df_tmp3['HIT_RESULT'] == "홈런"]['HIT_ANG_VER'], [25, 75])
        q1S_Ang, q3S_Ang = np.percentile(df_tmp3[df_tmp3['HIT_RESULT'] == "2루타"]['HIT_ANG_VER'], [25, 75])
        q1T_Ang, q3T_Ang = np.percentile(df_tmp3[df_tmp3['HIT_RESULT'] == "3루타"]['HIT_ANG_VER'], [25, 75])
        AngMax = np.max([q1H_Ang, q3H_Ang, q1S_Ang, q3S_Ang, q1T_Ang, q3T_Ang])
        AngMin = np.min([q1H_Ang, q3H_Ang, q1S_Ang, q3S_Ang, q1T_Ang, q3T_Ang])

        q1H_Vel, q3H_Vel = np.percentile(df_tmp3[df_tmp3['HIT_RESULT'] == "홈런"]['HIT_VEL'], [25, 75])
        q1S_Vel, q3S_Vel = np.percentile(df_tmp3[df_tmp3['HIT_RESULT'] == "2루타"]['HIT_VEL'], [25, 75])
        q1T_Vel, q3T_Vel = np.percentile(df_tmp3[df_tmp3['HIT_RESULT'] == "3루타"]['HIT_VEL'], [25, 75])
        VelMax = np.max([q1H_Vel, q3H_Vel, q1S_Vel, q3S_Vel, q1T_Vel, q3T_Vel])
        VelMin = np.min([q1H_Vel, q3H_Vel, q1S_Vel, q3S_Vel, q1T_Vel, q3T_Vel])

    
        condition = ((df_tmp2['HIT_VEL'] >= VelMin) & (df_tmp2['HIT_VEL'] <= VelMax)) & ((df_tmp2['HIT_ANG_VER'] >= AngMin) & (df_tmp2['HIT_ANG_VER'] <= AngMax))
        barrel_index = df_tmp2[condition].index
        df['barrel'].iloc[barrel_index] = True

    return df

In [65]:
df_add_barrel = barrel(df_all)
df_add_barrel.head()

,PCODE,GYEAR,G_ID,PIT_ID,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM,NAME,barrel
0,60100,2018,20180328HHNC0,180328_203348,HH,7,148.19,-4.7,땅볼아웃,140.56,마산,백창수,False
1,60100,2018,20180330SKHH0,180330_190656,HH,2,153.79,5.8,직선타,142.41,대전,백창수,False
2,60100,2018,20180331SKHH0,180331_174531,HH,2,148.65,-13.5,땅볼아웃,135.50,대전,백창수,False
3,60100,2018,20180331SKHH0,180331_192505,HH,7,151.85,-3.7,병살타,131.71,대전,백창수,False
4,60100,2018,20180331SKHH0,180331_200628,HH,9,154.12,19.0,플라이,141.85,대전,백창수,True


In [66]:
df_barrel = df_add_barrel[df_add_barrel['barrel']]

In [67]:
df_barrel['HIT_RESULT'].value_counts()

플라이      5934
2루타      3908
1루타      2598
홈런       2497
직선타       572
3루타       380
희생플라이     362
땅볼아웃        5
Name: HIT_RESULT, dtype: int64

In [68]:
barrel_ab = len(df_barrel[~(df_barrel['HIT_RESULT'] == "희생플라이")])
barrel_hit = df_barrel['HIT_RESULT'].value_counts()[['1루타', '2루타', '3루타', '홈런']].sum()
barrel_long_hit = df_barrel['HIT_RESULT'].value_counts()['1루타']*1 + df_barrel['HIT_RESULT'].value_counts()['2루타']*2 + \
    df_barrel['HIT_RESULT'].value_counts()['3루타']*3 + df_barrel['HIT_RESULT'].value_counts()['홈런']*4

In [69]:
#타율
barrel_avg =  barrel_hit / barrel_ab
#장타율
barrel_slg = barrel_long_hit / barrel_ab

print("배럴타구 타율 : ", barrel_avg ,"배럴타구 장타율 : ", barrel_slg)

배럴타구 타율 :  0.5903485592047314 배럴타구 장타율 :  1.3553542217188876


In [73]:
df_add_barrel['Date'] = df_add_barrel['G_ID'].str[:8]
df_add_barrel

,PCODE,GYEAR,G_ID,PIT_ID,T_ID,INN,HIT_VEL,HIT_ANG_VER,HIT_RESULT,PIT_VEL,STADIUM,NAME,barrel,Date
0,60100,2018,20180328HHNC0,180328_203348,HH,7,148.19,-4.7,땅볼아웃,140.56,마산,백창수,False,20180328
1,60100,2018,20180330SKHH0,180330_190656,HH,2,153.79,5.8,직선타,142.41,대전,백창수,False,20180330
2,60100,2018,20180331SKHH0,180331_174531,HH,2,148.65,-13.5,땅볼아웃,135.50,대전,백창수,False,20180331
3,60100,2018,20180331SKHH0,180331_192505,HH,7,151.85,-3.7,병살타,131.71,대전,백창수,False,20180331
4,60100,2018,20180331SKHH0,180331_200628,HH,9,154.12,19.0,플라이,141.85,대전,백창수,True,20180331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120740,79608,2021,20210709LTSS0,210709_223300,LT,11,141.40,25.8,플라이,135.88,대구,안치홍,True,20210709
120741,79608,2021,20210710LTSS0,210710_182636,LT,2,120.76,17.3,1루타,121.50,대구,안치홍,False,20210710
120742,79608,2021,20210710LTSS0,210710_205259,LT,4,156.86,11.4,2루타,126.40,대구,안치홍,True,20210710
120743,79608,2021,20210711LTSS0,210711_181318,LT,4,122.34,69.3,플라이,129.76,대구,안치홍,False,20210711


In [74]:
df_add_barrel.to_csv('../../dataset/배럴타구여부추가.csv', 
                    encoding='utf-8-sig', index=False)